In [1]:
import os, sys, time, glob
import warnings
import re
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import pandana as pdna
pd.options.mode.chained_assignment = None
import h5py
sys.path.append(os.path.join(os.getcwd(),r'..\..\..'))
# from input_configuration import base_year
import toml
config = toml.load(os.path.join(os.getcwd(),r'../../../configuration/input_configuration.toml'))

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
survey_path = '../../../inputs/base_year/survey'

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [3]:
pd.options.display.float_format = '{:0,.0f}'.format

In [4]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Congested network components by time of day
    df.columns

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    
    return df

In [5]:
output_dir = r'..\..\..\outputs\compare\RTP\person'

# Trips per Day by Resident
County, Center, Regional Geography

Youth, elderly, LEP, Race & Poverty for Region Work/Non-Work

In [6]:
pd.options.display.float_format = '{:0,.1f}'.format
df_trip = pd.read_csv(r'..\..\..\outputs\agg\dash\person_trips.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

df_trip['is_rgc'] = 0
df_trip.loc[df_trip['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

In [7]:
test = pd.read_csv(r'R:\e2projects_two\SoundCast\Inputs\db_inputs\parcel_2018_geography.csv')

In [8]:
test['GrowthCenterName'].value_counts()

Not in RGC                                  1273792
Seattle First Hill/Capitol Hill                2883
Tacoma Downtown                                2868
Duwamish                                       2013
Everett                                        1964
Seattle University Community                   1401
Seattle Downtown                               1348
Kent MIC                                        873
Tacoma Mall                                     861
Burien                                          802
Ballard-Interbay                                797
Bremerton                                       779
Cascade                                         755
SeaTac                                          743
Port of Tacoma                                  732
Lakewood                                        699
Paine Field / Boeing Everett                    681
Renton                                          674
Puyallup Downtown                               668
Seattle Upto

In [9]:
df_trip['hh_rgc'].unique()

array(['Kirkland Totem Lake', 'Not in RGC', 'Auburn', 'Bellevue',
       'Bothell Canyon Park', 'Bremerton', 'Burien', 'Everett',
       'Federal Way', 'Greater Downtown Kirkland', 'Kent', 'Lakewood',
       'Lynnwood', 'Puyallup Downtown', 'Puyallup South Hill',
       'Redmond Downtown', 'Redmond-Overlake', 'Renton', 'SeaTac',
       'Seattle Downtown', 'Seattle First Hill/Capitol Hill',
       'Seattle Northgate', 'Seattle South Lake Union',
       'Seattle University Community', 'Seattle Uptown', 'Silverdale',
       'Tacoma Downtown', 'Tacoma Mall', 'Tukwila', 'University Place'],
      dtype=object)

### County and Region

In [10]:
def trips_per_day(geog): 
    df1 = df_person.groupby(geog).sum()[['psexpfac']]
    df2 = df_trip.groupby(geog).sum()[['trexpfac']]
    df = df1.merge(df2, left_index=True, right_index=True)
    df.loc['Region', 'psexpfac'] = df_person.sum()['psexpfac']
    df.loc['Region', 'trexpfac'] = df_trip.sum()['trexpfac']
    df['Total Trips per Day'] = df['trexpfac']/df['psexpfac']

    df_trip.loc[df_trip['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
    df_trip.loc[df_trip['dpurp'] == 'Work', 'trip_type'] = 'Work'
    for purp in ['Work','Non-Work']:
        _df = df_trip[df_trip['trip_type'] == purp].groupby(geog).sum()[['trexpfac']]
        _df.rename(columns={'trexpfac': purp + ' Trips'}, inplace=True)

        # Add regional total to bottom of table
        _df.loc['Region', purp + ' Trips'] = df_trip[df_trip['trip_type'] == purp].sum()['trexpfac']
        df = df.merge(_df, left_index=True, right_index=True)

    df['Work Trips per Day'] = df['Work Trips']/df['psexpfac']
    df['Non-Work Trips per Day'] = df['Non-Work Trips']/df['psexpfac']
    df[['Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']]
    
    return df[['Work Trips per Day', 'Non-Work Trips per Day','Total Trips per Day']]

In [11]:
df = trips_per_day('hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_county.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_county,,,
King,0.7,3.4,4.0
Kitsap,0.9,5.7,6.6
Pierce,0.8,4.7,5.5
Snohomish,0.9,4.7,5.6
Region,0.7,3.8,4.6


### Regional Growth Centers

In [12]:
df = trips_per_day('is_rgc')
# df.index = ['Not in RGC', 'In RGC','Region']
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
is_rgc,,,
0,0.7,3.9,4.7
1,0.8,3.1,3.9
Region,0.7,3.8,4.6


In [13]:
df = trips_per_day('hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_rgc.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rgc,,,
Auburn,0.5,3.3,3.8
Bellevue,0.8,3.2,4.0
Bothell Canyon Park,0.8,3.2,3.9
Bremerton,0.6,3.3,3.9
Burien,0.7,3.2,3.8
Everett,0.6,3.3,3.9
Federal Way,0.6,3.6,4.2
Greater Downtown Kirkland,0.7,3.1,3.8
Kent,0.5,3.2,3.8


### Regional Geographies

In [14]:
df = trips_per_day('hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_reg_geog.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rg_proposed,,,
CitiesTowns,0.5,3.1,3.7
Core,0.6,3.2,3.8
HCT,1.1,6.3,7.4
Metro,0.7,3.2,3.9
Region,0.7,3.8,4.6


### Equity Geographies

In [15]:
# Summary by equity geographies
# Calculate person totals
results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        val = df_person[df_person[equity_geog+geog_type] == 1].sum()['psexpfac']
        results_df.loc[equity_geog+geog_type,'psexpfac'] = val
        
# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        for purp in ['Non-Work','Work']:
            _df = df_trip[df_trip['trip_type'] == purp].groupby(equity_geog+geog_type).sum()[['trexpfac']]
            if 1 in _df.index:
                if geog_type == '_reg' or equity_geog in list_50:
                    _rate_df = pd.DataFrame([purp,
                                         equity_geog+geog_type,
                                         _df.loc[1].trexpfac/results_df.loc[equity_geog+geog_type].psexpfac]).T
                trip_rate_df = trip_rate_df.append(_rate_df) 
trip_rate_df.columns = ['Trip Type','Equity Group','Trip Rate']
df = trip_rate_df.pivot_table(columns='Trip Type', index='Equity Group', values='Trip Rate', aggfunc='sum')
df['Total Trips per Day'] = df['Non-Work'] + df['Work']
df.rename(columns={'Non-Work': 'Non-Work Trips per Day', 'Work': 'Work Trips per Day'}, inplace=True)
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_equity.csv'))
HTML(df[['Equity Group','Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']].to_html(index=False))

Equity Group,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_disability_reg,0.7,3.9,4.6
hh_elderly_reg,2.0,3.7,5.7
hh_poverty_50,0.6,3.6,4.2
hh_poverty_reg,0.7,3.9,4.6
hh_racial_50,0.7,3.8,4.5
hh_racial_reg,0.8,4.0,4.7
hh_youth_reg,0.8,4.2,5.0


In [16]:
# Summary by equity geographies
# Calculate person totals
results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        val = df_person[df_person[equity_geog+geog_type] == 0].sum()['psexpfac']
        results_df.loc[equity_geog+geog_type,'psexpfac'] = val
        
# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        for purp in ['Non-Work','Work']:
            _df = df_trip[df_trip['trip_type'] == purp].groupby(equity_geog+geog_type).sum()[['trexpfac']]
            if 1 in _df.index:
                if geog_type == '_reg' or equity_geog in list_50:
                    _rate_df = pd.DataFrame([purp,
                                         equity_geog+geog_type,
                                         _df.loc[1].trexpfac/results_df.loc[equity_geog+geog_type].psexpfac]).T
                trip_rate_df = trip_rate_df.append(_rate_df) 
trip_rate_df.columns = ['Trip Type','Equity Group','Trip Rate']
df = trip_rate_df.pivot_table(columns='Trip Type', index='Equity Group', values='Trip Rate', aggfunc='sum')
df['Total Trips per Day'] = df['Non-Work'] + df['Work']

df.rename(columns={'Non-Work': 'Non-Work Trips per Day', 'Work': 'Work Trips per Day'}, inplace=True)
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df['Equity Group'] = ['NOT in '+i for i in df['Equity Group']]
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_equity.csv'))
HTML(df[['Equity Group','Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']].to_html(index=False))

Equity Group,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
NOT in hh_disability_reg,0.5,2.9,3.5
NOT in hh_elderly_reg,1.7,3.1,4.9
NOT in hh_poverty_50,0.0,0.1,0.1
NOT in hh_poverty_reg,0.5,2.8,3.3
NOT in hh_racial_50,0.4,1.8,2.2
NOT in hh_racial_reg,0.8,4.4,5.2
NOT in hh_youth_reg,0.6,3.5,4.1


# Miles Driven per Day by Resident
County, Center, Regional Geography

Youth, elderly, LEP, Race & Poverty for Region Work/Non-Work

In [17]:
pd.options.display.float_format = '{:0,.1f}'.format
df_vmt = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vmt.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

df_vmt['is_rgc'] = 0
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)]

def vmt_per_person(df_vmt, df_person, geog):
    _df_vmt = df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    
    return df[['Average Miles per Person']]

### County

In [18]:
df = vmt_per_person(df_vmt, df_person, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_county.csv')); incr+=1
df

,Average Miles per Person
hh_county,
King,13.1
Kitsap,10.7
Pierce,12.8
Snohomish,14.6
Region,13.2


### Regional Growth Centers

In [19]:
df = vmt_per_person(df_vmt, df_person, 'is_rgc')
df.index = ['Not in RGC', 'In RGC','Region']
df

,Average Miles per Person
Not in RGC,13.7
In RGC,7.9
Region,13.2


In [20]:
df = vmt_per_person(df_vmt, df_person, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_rgc.csv')); incr+=1
df

,Average Miles per Person
hh_rgc,
Auburn,13.4
Bellevue,9.7
Bothell Canyon Park,14.8
Bremerton,9.3
Burien,13.2
Everett,13.1
Federal Way,14.6
Greater Downtown Kirkland,15.0
Kent,11.9


### Regional Geography

In [21]:
df = vmt_per_person(df_vmt, df_person, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_reg_geog.csv')); incr+=1
df

,Average Miles per Person
hh_rg_proposed,
CitiesTowns,17.3
Core,14.5
HCT,14.4
Metro,10.6
Region,13.2


### Equity Geographies
Average Miles per Person (by residence inside or outside equity geography) 

In [22]:
result_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = vmt_per_person(df_vmt, df_person, col)
            if 1 in df.index:
                df = df.loc[1]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df = result_df[~result_df.index.duplicated()]
df_in = result_df.copy()

result_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = vmt_per_person(df_vmt, df_person, col)
            if 0 in df.index:
                df = df.loc[0]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df = result_df[~result_df.index.duplicated()]
# result_df.index = ['NOT in '+i for i in result_df.index]


df_out = result_df.copy()

df = df_in.merge(df_out, left_index=True, right_index=True)
df.columns = ['Inside Equity Geography',  'Outside Equity Geography']
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_equity.csv')); incr+=1
df

,Inside Equity Geography,Outside Equity Geography
source,,
hh_youth_reg,14.6,12.1
hh_elderly_reg,13.2,13.2
hh_english_reg,13.2,13.2
hh_racial_reg,13.0,13.5
hh_racial_50,13.1,13.3
hh_poverty_reg,12.3,13.8
hh_poverty_50,9.6,13.3
hh_disability_reg,12.8,13.6


# Miles Walking and Biking per Day by Resident
County, Center, Regional Geography

Youth, elderly, LEP, Race & Poverty for Region Work/Non-Work

In [23]:
df_vht = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vht.csv')
df_hh = pd.read_csv(r'..\..\..\outputs\agg\dash\hh_geog.csv')



def walk_bike_per_person_time(df_vmt, df_person, geog):
    
    results_df = pd.DataFrame()
    for mode in ['Walk','Bike']:
        
        _df_vmt = df_vmt[df_vmt['mode'] == mode]
        _df_vmt = _df_vmt.groupby(geog).sum()[['travtime_wt']]
        _df_hh = df_hh.groupby(geog).sum()[['hhexpfac']]

        df = _df_vmt.merge(_df_hh, left_index=True, right_index=True)
        df.loc['Region',:] = df.sum(axis=0)
        df['Average Miles per Person'] = df['travtime_wt']/df['hhexpfac']
        results_df[mode] = df['Average Miles per Person']
    
    _df_vmt = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])]
    _df_vmt = _df_vmt.groupby(geog).sum()[['travtime_wt']]
    _df_hh = df_hh.groupby(geog).sum()[['hhexpfac']]

    df = _df_vmt.merge(_df_hh, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travtime_wt']/df['hhexpfac']
    results_df['Walk and Bike'] = df['Average Miles per Person']
    
    return results_df

df = walk_bike_per_person_time(df_vht, df_hh, 'hh_county')
# df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_county.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_county,,,
King,21.9,3.1,25.0
Kitsap,20.5,2.1,22.6
Pierce,19.6,2.7,22.2
Snohomish,19.9,2.9,22.8
Region,21.2,3.0,24.2


In [24]:
# df = pd.read_csv(r'..\..\..\outputs\agg\dash\trip_distance.csv')
# _df = df[df['mode'] == 'Bike']
# (_df['travdist_bin']*_df['trexpfac']).sum()/_df['trexpfac'].sum()

In [25]:
# _df = df[df['mode'] == 'Walk']
# (_df['travdist_bin']*_df['trexpfac']).sum()/_df['trexpfac'].sum()

In [26]:
df_vmt = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vmt.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

df_vmt['is_rgc'] = 0
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1


def walk_bike_per_person(df_vmt, df_person, geog):
    
    results_df = pd.DataFrame()
    for mode in ['Walk','Bike']:
        
        _df_vmt = df_vmt[df_vmt['mode'] == mode]
        _df_vmt = _df_vmt.groupby(geog).sum()[['travdist_wt']]
        _df_person = df_person.groupby(geog).sum()[['psexpfac']]

        df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
        df.loc['Region',:] = df.sum(axis=0)
        df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
        results_df[mode] = df['Average Miles per Person']
    
    _df_vmt = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])]
    _df_vmt = _df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    results_df['Walk and Bike'] = df['Average Miles per Person']
    
    return results_df

### County

In [27]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_county.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_county,,,
King,0.5,0.2,0.7
Kitsap,0.4,0.1,0.6
Pierce,0.4,0.2,0.6
Snohomish,0.4,0.2,0.6
Region,0.4,0.2,0.6


### Regional Growth Center 

In [28]:
df = walk_bike_per_person(df_vmt, df_person, 'is_rgc')
df.index = ['Not in RGC', 'Inside RGC', 'Region']
df

,Walk,Bike,Walk and Bike
Not in RGC,0.4,0.2,0.6
Inside RGC,0.8,0.3,1.1
Region,0.4,0.2,0.6


In [29]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_rgc.csv'))
df

,Walk,Bike,Walk and Bike
hh_rgc,,,
Auburn,0.5,0.4,0.9
Bellevue,0.7,0.3,0.9
Bothell Canyon Park,0.5,0.3,0.7
Bremerton,0.6,0.2,0.8
Burien,0.5,0.2,0.7
Everett,0.6,0.3,0.9
Federal Way,0.8,0.4,1.2
Greater Downtown Kirkland,0.4,0.3,0.8
Kent,0.4,0.3,0.7


### Regional Geography

In [30]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_reg_geog.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_rg_proposed,,,
CitiesTowns,0.3,0.1,0.5
Core,0.4,0.2,0.6
HCT,0.4,0.2,0.5
Metro,0.6,0.2,0.8
Region,0.4,0.2,0.6


### Equity Geographies

In [31]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = walk_bike_per_person(df_vmt, df_person, col)
            if 1 in df.index:
                df = df.loc[1]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_equity.csv')); incr+=1
result_df

,Bike,Walk,Walk and Bike
source,,,
hh_youth_reg,0.2,0.4,0.5
hh_elderly_reg,0.2,0.4,0.6
hh_english_reg,0.2,0.4,0.6
hh_english_reg,0.2,0.4,0.6
hh_racial_reg,0.2,0.4,0.6
hh_racial_50,0.2,0.4,0.6
hh_poverty_reg,0.2,0.5,0.7
hh_poverty_50,0.3,0.6,0.9
hh_disability_reg,0.2,0.5,0.6


In [32]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = walk_bike_per_person(df_vmt, df_person, col)
            if 0 in df.index:
                df = df.loc[0]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.index = ['NOT in '+i for i in result_df.index]
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_equity.csv')); incr+=1
result_df

,Bike,Walk,Walk and Bike
NOT in hh_youth_reg,0.2,0.5,0.7
NOT in hh_elderly_reg,0.2,0.5,0.6
NOT in hh_english_reg,0.2,0.4,0.6
NOT in hh_racial_reg,0.2,0.4,0.6
NOT in hh_racial_50,0.2,0.4,0.6
NOT in hh_poverty_reg,0.2,0.4,0.6
NOT in hh_poverty_50,0.2,0.4,0.6
NOT in hh_disability_reg,0.2,0.4,0.6


# Jobs Accessible within 45 Minutes of Transit

In [33]:
pd.options.display.float_format = '{:0,.0f}'.format
df = pd.read_csv(r'..\..\..\outputs\access\transit_jobs_access.csv'); incr+=1
df_lu = pd.read_csv(r'..\..\..\outputs\landuse\parcels_urbansim.txt',
                   sep='\s+', usecols=['parcelid','emptot_p'])
tot_jobs = df_lu['emptot_p'].sum()

### Region

In [34]:
_df = df[df['geography_group'] == 'region']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_region.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Jobs within 45-minute Transit Commute,% Total Jobs
"137,679",6.3%


### County

In [35]:
_df = df[df['geography_group'] == 'CountyName']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_county.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
})
display(HTML(output))

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
King,"232,713",10.6%
Kitsap,"6,470",0.3%
Outside Region,0,0.0%
Pierce,"19,719",0.9%
Snohomish,"26,928",1.2%


### Regional Growth Center

In [36]:
_df = df[df['geography_group'] == 'GrowthCenterName']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_rgc.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
})
display(HTML(output))

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
Auburn,"68,258",3.1%
Bellevue,"483,710",22.1%
Bothell Canyon Park,"39,684",1.8%
Bremerton,"34,689",1.6%
Burien,"126,449",5.8%
Everett,"59,043",2.7%
Federal Way,"101,679",4.6%
Greater Downtown Kirkland,"258,274",11.8%
Issaquah,0,0.0%


### Regional Geography

In [37]:
_df = df[df['geography_group'] == 'rg_proposed']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_reg_geog.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
})
display(HTML(output))

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
CitiesTowns,"10,655",0.5%
Core,"61,220",2.8%
HCT,"37,257",1.7%
Metro,"338,273",15.4%
UU,"9,855",0.4%


### Equity Geographies

In [38]:
geo_list = []
equity_geogs = ['youth','elderly','english','racial','poverty','disability']
for equity_geog in equity_geogs:
    for geog_type in ['_geog_vs_reg_total','_geog_vs_50_percent']:
        if geog_type == '_geog_vs_reg_total' or equity_geog in ['poverty','racial']:
            geo_list.append(equity_geog+geog_type)

In [39]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography'] == '1')]
    result_df = result_df.append(_df[['geography_group', 'value']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
# result_df = result_df['']
result_df['% Total Jobs'] = result_df['Jobs within 45-minute Transit Commute']/tot_jobs
output = result_df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 45-minute Transit Commute,% Total Jobs
youth_geog_vs_reg_total,"46,458",2.1%
elderly_geog_vs_reg_total,"106,668",4.9%
racial_geog_vs_reg_total,"160,338",7.3%
racial_geog_vs_50_percent,"168,879",7.7%
poverty_geog_vs_reg_total,"140,228",6.4%
poverty_geog_vs_50_percent,"309,594",14.1%
disability_geog_vs_reg_total,"111,754",5.1%


In [40]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography'] == '0')]
    result_df = result_df.append(_df[['geography_group', 'value']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
# result_df = result_df['']
result_df['% Total Jobs'] = result_df['Jobs within 45-minute Transit Commute']/tot_jobs
result_df['Equity Group'] = ['NOT in '+i for i in result_df['Equity Group']]
output = result_df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 45-minute Transit Commute,% Total Jobs
NOT in youth_geog_vs_reg_total,"217,128",9.9%
NOT in elderly_geog_vs_reg_total,"167,029",7.6%
NOT in english_geog_vs_reg_total,"137,679",6.3%
NOT in racial_geog_vs_reg_total,"116,940",5.3%
NOT in racial_geog_vs_50_percent,"125,659",5.7%
NOT in poverty_geog_vs_reg_total,"135,919",6.2%
NOT in poverty_geog_vs_50_percent,"133,959",6.1%
NOT in disability_geog_vs_reg_total,"159,985",7.3%


# Average Jobs Accessible within 1 Mile Walk and 3 Mile Bike
Note that this is not using the bike network, but is instead using the all-streets network.
Average accessible jobs are weighted averages based on parcel household population.

In [41]:
df = pd.read_csv(r'..\..\..\outputs\access\walk_bike_jobs_access.csv')

### Regional Average

In [42]:
_df = df[df['geography_group'] == 'region']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
"11,229","55,709",0.5%,2.5%


### Home County

In [43]:
_df = df[df['geography_group'] == 'CountyName']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_county.csv')); incr+=1
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
})
display(HTML(output))

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography_value,,,,
King,"18,645","87,376",0.9%,4.0%
Kitsap,"1,178","7,596",0.1%,0.3%
Outside Region,3,35,0.0%,0.0%
Pierce,"2,503","18,420",0.1%,0.8%
Snohomish,"1,999","18,090",0.1%,0.8%


### Regional Growth Center

In [44]:
_df = df[df['geography_group'] == 'GrowthCenterName']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_rgc.csv')); incr+=1
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
})
display(HTML(output))

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography_value,,,,
Auburn,"10,412","40,870",0.5%,1.9%
Bellevue,"60,057","111,465",2.7%,5.1%
Bothell Canyon Park,"8,932","22,686",0.4%,1.0%
Bremerton,"11,523","29,473",0.5%,1.3%
Burien,"5,130","15,290",0.2%,0.7%
Everett,"15,035","35,634",0.7%,1.6%
Federal Way,"6,098","26,418",0.3%,1.2%
Greater Downtown Kirkland,"11,938","37,240",0.5%,1.7%
Issaquah,NaN,NaN,NaN,NaN


### Regional Geography

In [45]:
_df = df[df['geography_group'] == 'rg_proposed']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_reg_geog.csv')); incr+=1
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
})
display(HTML(output))

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography_value,,,,
CitiesTowns,972,"7,819",0.0%,0.4%
Core,"3,332","29,061",0.2%,1.3%
HCT,"1,488","16,032",0.1%,0.7%
Metro,"29,646","131,802",1.4%,6.0%
UU,415,"6,747",0.0%,0.3%


### Equity Geographies

In [46]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography_value'] == '1')]
    result_df = result_df.append(_df[['geography_group', 'jobs_1_mile_walk','jobs_3_mile_bike']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 
                          'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                        'jobs_3_mile_bike':  'Jobs within 3-mile Bike'}, inplace=True)

result_df['% Total Jobs (1-mile walk)'] = result_df['Jobs within 1-mile Walk']/tot_jobs
result_df['% Total Jobs (3-mile bike)'] = result_df['Jobs within 3-mile Bike']/tot_jobs
output = result_df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
youth_geog_vs_reg_total,"1,730","19,548",0.1%,0.9%
elderly_geog_vs_reg_total,"10,001","44,721",0.5%,2.0%
racial_geog_vs_reg_total,"17,866","70,396",0.8%,3.2%
racial_geog_vs_50_percent,"18,404","72,090",0.8%,3.3%
poverty_geog_vs_reg_total,"13,978","58,248",0.6%,2.7%
poverty_geog_vs_50_percent,"34,106","107,169",1.6%,4.9%
disability_geog_vs_reg_total,"13,877","52,890",0.6%,2.4%


In [47]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography_value'] == '0')]
    result_df = result_df.append(_df[['geography_group', 'jobs_1_mile_walk','jobs_3_mile_bike']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 
                          'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                        'jobs_3_mile_bike':  'Jobs within 3-mile Bike'}, inplace=True)

result_df['% Total Jobs (1-mile walk)'] = result_df['Jobs within 1-mile Walk']/tot_jobs
result_df['% Total Jobs (3-mile bike)'] = result_df['Jobs within 3-mile Bike']/tot_jobs
result_df['Equity Group'] = ['NOT in '+i for i in result_df['Equity Group']]
output = result_df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
NOT in youth_geog_vs_reg_total,"19,503","87,203",0.9%,4.0%
NOT in elderly_geog_vs_reg_total,"12,392","66,109",0.6%,3.0%
NOT in english_geog_vs_reg_total,"11,229","55,709",0.5%,2.5%
NOT in racial_geog_vs_reg_total,"5,155","42,266",0.2%,1.9%
NOT in racial_geog_vs_50_percent,"8,465","49,398",0.4%,2.3%
NOT in poverty_geog_vs_reg_total,"9,330","53,955",0.4%,2.5%
NOT in poverty_geog_vs_50_percent,"10,734","54,596",0.5%,2.5%
NOT in disability_geog_vs_reg_total,"8,952","58,134",0.4%,2.7%
